# Image Processing Assignments - Frequency Domain Filtering

This notebook covers the implementation of frequency domain filtering techniques as outlined in Assignment No. 3 (Smoothing/Lowpass Filtering) and Assignment No. 4 (Sharpening/Highpass Filtering).

**Instructions for Submission:**

1. Implement all activities using Python.
2. Show the code and the output results for each activity (original image + filtered/enhanced image).
3. Ensure the output is neat and clearly visible.

## Assignment No. 3: Frequency Domain Smoothing (Lowpass Filtering)

This section focuses on applying and comparing frequency domain smoothing filters on a grayscale image. The goal is to understand the behavior of each filter in terms of noise reduction and image blurring by examining the filtered outputs and their corresponding magnitude spectra.

**Common Instructions for all Lowpass Filters:**

* **Cutoff Frequency ($D_0$):** Use $D_0 = 50$.
* **Inverse FFT:** Perform inverse FFT to reconstruct the image.
* **Display and Compare:** Original image, filtered outputs, and their corresponding magnitude spectra.

### Ideal Lowpass Filter (ILPF)

**Description:**
The Ideal Lowpass Filter (ILPF) allows all frequencies within a specified cutoff distance ($D_0$) from the origin to pass unimpeded, while completely cutting off all frequencies outside this circle. It creates a sharp boundary in the frequency domain, which can lead to ringing artifacts (Gibbs phenomenon) in the spatial domain.

**Task:**
Implement and apply the Ideal Lowpass Filter in the frequency domain. Show the code, original image, filtered output, and its magnitude spectrum.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image
# Replace 'your_image_path.jpg' with the actual path to your image file
image = cv2.imread('image.jpg', 0) # Load as grayscale

if image is None:
    print("Error: Could not load image. Please check the file path.")
else:
    # Get image dimensions
    rows, cols = image.shape

    # Pad the image to optimal size for FFT (power of 2)
    nrows = cv2.getOptimalDFTSize(rows)
    ncols = cv2.getOptimalDFTSize(cols)
    padded_image = np.zeros((nrows, ncols), dtype=np.float32)
    padded_image[:rows, :cols] = image.astype(np.float32)

    # Perform 2D Discrete Fourier Transform (DFT)
    dft = cv2.dft(padded_image, flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    # Create a meshgrid for filter creation
    center_x, center_y = ncols // 2, nrows // 2
    x = np.arange(ncols) - center_x
    y = np.arange(nrows) - center_y
    X, Y = np.meshgrid(x, y)
    D = np.sqrt(X**2 + Y**2) # Euclidean distance from center

    # Ideal Lowpass Filter (ILPF)
    D0 = 50 # Cutoff frequency
    H_ilpf = np.zeros((nrows, ncols), dtype=np.float32)
    H_ilpf[D <= D0] = 1

    # Apply the filter in the frequency domain
    filtered_dft_shift_ilpf = dft_shift * H_ilpf[:, :, np.newaxis] # Apply to both real and imaginary parts

    # Compute magnitude spectrum of the filtered image
    magnitude_spectrum_filtered_ilpf = 20 * np.log(cv2.magnitude(filtered_dft_shift_ilpf[:, :, 0], filtered_dft_shift_ilpf[:, :, 1]) + 1)

    # Perform inverse DFT
    f_ishift_ilpf = np.fft.ifftshift(filtered_dft_shift_ilpf)
    img_back_ilpf = cv2.idft(f_ishift_ilpf, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
    img_back_ilpf = img_back_ilpf[:rows, :cols] # Crop back to original size
    img_back_ilpf = np.clip(img_back_ilpf, 0, 255).astype(np.uint8)

    # Compute magnitude spectrum of the original image for comparison
    magnitude_spectrum_original = 20 * np.log(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]) + 1)

    # Display results
    plt.figure(figsize=(15, 5))
    plt.subplot(131), plt.imshow(image, cmap='gray'), plt.title('Original Image')
    plt.subplot(132), plt.imshow(img_back_ilpf, cmap='gray'), plt.title(f'ILPF Filtered (D0={D0})')
    plt.subplot(133), plt.imshow(magnitude_spectrum_filtered_ilpf, cmap='gray'), plt.title('ILPF Magnitude Spectrum')
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(7, 5))
    plt.imshow(magnitude_spectrum_original, cmap='gray'), plt.title('Original Magnitude Spectrum')
    plt.tight_layout()
    plt.show()


### Butterworth Lowpass Filter (BLPF)

**Description:**
The Butterworth Lowpass Filter (BLPF) has a smooth transition between passed and blocked frequencies, which helps to reduce ringing artifacts compared to the Ideal filter. The steepness of the transition is controlled by the filter order ($n$). A higher order results in a sharper transition.

**Task:**
Implement and apply the Butterworth Lowpass Filter with order $n=2$ in the frequency domain. Show the code, original image, filtered output, and its magnitude spectrum.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image
# Replace 'your_image_path.jpg' with the actual path to your image file
image = cv2.imread('image.jpg', 0) # Load as grayscale

if image is None:
    print("Error: Could not load image. Please check the file path.")
else:
    # Get image dimensions
    rows, cols = image.shape

    # Pad the image to optimal size for FFT (power of 2)
    nrows = cv2.getOptimalDFTSize(rows)
    ncols = cv2.getOptimalDFTSize(cols)
    padded_image = np.zeros((nrows, ncols), dtype=np.float32)
    padded_image[:rows, :cols] = image.astype(np.float32)

    # Perform 2D Discrete Fourier Transform (DFT)
    dft = cv2.dft(padded_image, flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    # Create a meshgrid for filter creation
    center_x, center_y = ncols // 2, nrows // 2
    x = np.arange(ncols) - center_x
    y = np.arange(nrows) - center_y
    X, Y = np.meshgrid(x, y)
    D = np.sqrt(X**2 + Y**2) # Euclidean distance from center

    # Butterworth Lowpass Filter (BLPF)
    D0 = 50 # Cutoff frequency
    n = 2   # Filter order
    H_blpf = 1 / (1 + (D / D0)**(2 * n))

    # Apply the filter in the frequency domain
    filtered_dft_shift_blpf = dft_shift * H_blpf[:, :, np.newaxis]

    # Compute magnitude spectrum of the filtered image
    magnitude_spectrum_filtered_blpf = 20 * np.log(cv2.magnitude(filtered_dft_shift_blpf[:, :, 0], filtered_dft_shift_blpf[:, :, 1]) + 1)

    # Perform inverse DFT
    f_ishift_blpf = np.fft.ifftshift(filtered_dft_shift_blpf)
    img_back_blpf = cv2.idft(f_ishift_blpf, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
    img_back_blpf = img_back_blpf[:rows, :cols] # Crop back to original size
    img_back_blpf = np.clip(img_back_blpf, 0, 255).astype(np.uint8)

    # Display results
    plt.figure(figsize=(15, 5))
    plt.subplot(131), plt.imshow(image, cmap='gray'), plt.title('Original Image')
    plt.subplot(132), plt.imshow(img_back_blpf, cmap='gray'), plt.title(f'BLPF Filtered (D0={D0}, n={n})')
    plt.subplot(133), plt.imshow(magnitude_spectrum_filtered_blpf, cmap='gray'), plt.title('BLPF Magnitude Spectrum')
    plt.tight_layout()
    plt.show()


### Gaussian Lowpass Filter (GLPF)

**Description:**
The Gaussian Lowpass Filter (GLPF) provides the smoothest transition in the frequency domain, which translates to no ringing artifacts in the spatial domain. Its shape is defined by the standard deviation, which is related to the cutoff frequency ($D_0$). It is often preferred for its smooth blurring characteristics.

**Task:**
Implement and apply the Gaussian Lowpass Filter in the frequency domain. Show the code, original image, filtered output, and its magnitude spectrum.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image
# Replace 'your_image_path.jpg' with the actual path to your image file
image = cv2.imread('image.jpg', 0) # Load as grayscale

if image is None:
    print("Error: Could not load image. Please check the file path.")
else:
    # Get image dimensions
    rows, cols = image.shape

    # Pad the image to optimal size for FFT (power of 2)
    nrows = cv2.getOptimalDFTSize(rows)
    ncols = cv2.getOptimalDFTSize(cols)
    padded_image = np.zeros((nrows, ncols), dtype=np.float32)
    padded_image[:rows, :cols] = image.astype(np.float32)

    # Perform 2D Discrete Fourier Transform (DFT)
    dft = cv2.dft(padded_image, flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    # Create a meshgrid for filter creation
    center_x, center_y = ncols // 2, nrows // 2
    x = np.arange(ncols) - center_x
    y = np.arange(nrows) - center_y
    X, Y = np.meshgrid(x, y)
    D = np.sqrt(X**2 + Y**2) # Euclidean distance from center

    # Gaussian Lowpass Filter (GLPF)
    D0 = 50 # Cutoff frequency (standard deviation)
    H_glpf = np.exp(-(D**2) / (2 * D0**2))

    # Apply the filter in the frequency domain
    filtered_dft_shift_glpf = dft_shift * H_glpf[:, :, np.newaxis]

    # Compute magnitude spectrum of the filtered image
    magnitude_spectrum_filtered_glpf = 20 * np.log(cv2.magnitude(filtered_dft_shift_glpf[:, :, 0], filtered_dft_shift_glpf[:, :, 1]) + 1)

    # Perform inverse DFT
    f_ishift_glpf = np.fft.ifftshift(filtered_dft_shift_glpf)
    img_back_glpf = cv2.idft(f_ishift_glpf, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
    img_back_glpf = img_back_glpf[:rows, :cols] # Crop back to original size
    img_back_glpf = np.clip(img_back_glpf, 0, 255).astype(np.uint8)

    # Display results
    plt.figure(figsize=(15, 5))
    plt.subplot(131), plt.imshow(image, cmap='gray'), plt.title('Original Image')
    plt.subplot(132), plt.imshow(img_back_glpf, cmap='gray'), plt.title(f'GLPF Filtered (D0={D0})')
    plt.subplot(133), plt.imshow(magnitude_spectrum_filtered_glpf, cmap='gray'), plt.title('GLPF Magnitude Spectrum')
    plt.tight_layout()
    plt.show()


## Assignment No. 4: Frequency Domain Sharpening (Highpass Filtering)

This section focuses on applying and analyzing sharpening filters in the frequency domain. High-pass filtering enhances edges and details by allowing high-frequency components to pass through while attenuating low-frequency components.

**Common Instructions for all Highpass Filters:**

* **Image Source:** Use the same grayscale image as in Assignment 03.
* **Cutoff Frequency ($D_0$):** Use a common cutoff frequency $D_0 = 50$.
* **Inverse FFT:** Perform inverse FFT to reconstruct the image.
* **Display and Compare:** Original image, sharpened outputs, and their corresponding magnitude spectra.

### Ideal Highpass Filter (IHPF)

**Description:**
The Ideal Highpass Filter (IHPF) is the inverse of the ILPF; it blocks all frequencies within a specified cutoff distance ($D_0$) and passes all others. Similar to its lowpass counterpart, the sharp transition can introduce ringing artifacts in the spatial domain, which appear as oscillations near edges.

**Task:**
Implement and apply the Ideal Highpass Filter in the frequency domain. Show the code, original image, sharpened output, and its magnitude spectrum.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image
# Replace 'your_image_path.jpg' with the actual path to your image file
image = cv2.imread('image.jpg', 0) # Load as grayscale

if image is None:
    print("Error: Could not load image. Please check the file path.")
else:
    # Get image dimensions
    rows, cols = image.shape

    # Pad the image to optimal size for FFT (power of 2)
    nrows = cv2.getOptimalDFTSize(rows)
    ncols = cv2.getOptimalDFTSize(cols)
    padded_image = np.zeros((nrows, ncols), dtype=np.float32)
    padded_image[:rows, :cols] = image.astype(np.float32)

    # Perform 2D Discrete Fourier Transform (DFT)
    dft = cv2.dft(padded_image, flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    # Create a meshgrid for filter creation
    center_x, center_y = ncols // 2, nrows // 2
    x = np.arange(ncols) - center_x
    y = np.arange(nrows) - center_y
    X, Y = np.meshgrid(x, y)
    D = np.sqrt(X**2 + Y**2) # Euclidean distance from center

    # Ideal Highpass Filter (IHPF)
    D0 = 50 # Cutoff frequency
    H_ihpf = np.ones((nrows, ncols), dtype=np.float32)
    H_ihpf[D <= D0] = 0 # Block frequencies within D0

    # Apply the filter in the frequency domain
    filtered_dft_shift_ihpf = dft_shift * H_ihpf[:, :, np.newaxis]

    # Compute magnitude spectrum of the filtered image
    magnitude_spectrum_filtered_ihpf = 20 * np.log(cv2.magnitude(filtered_dft_shift_ihpf[:, :, 0], filtered_dft_shift_ihpf[:, :, 1]) + 1)

    # Perform inverse DFT
    f_ishift_ihpf = np.fft.ifftshift(filtered_dft_shift_ihpf)
    img_back_ihpf = cv2.idft(f_ishift_ihpf, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
    img_back_ihpf = img_back_ihpf[:rows, :cols] # Crop back to original size
    img_back_ihpf = np.clip(img_back_ihpf, 0, 255).astype(np.uint8)

    # Display results
    plt.figure(figsize=(15, 5))
    plt.subplot(131), plt.imshow(image, cmap='gray'), plt.title('Original Image')
    plt.subplot(132), plt.imshow(img_back_ihpf, cmap='gray'), plt.title(f'IHPF Sharpened (D0={D0})')
    plt.subplot(133), plt.imshow(magnitude_spectrum_filtered_ihpf, cmap='gray'), plt.title('IHPF Magnitude Spectrum')
    plt.tight_layout()
    plt.show()


### Butterworth Highpass Filter (BHPF)

**Description:**
The Butterworth Highpass Filter (BHPF) offers a smooth transition between blocked and passed frequencies, reducing ringing artifacts commonly seen with ideal filters. Its order ($n$) controls the sharpness of this transition. A higher order results in a steeper cutoff, more closely resembling an ideal filter.

**Task:**
Implement and apply the Butterworth Highpass Filter with order $n=2$ in the frequency domain. Show the code, original image, sharpened output, and its magnitude spectrum.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image
# Replace 'your_image_path.jpg' with the actual path to your image file
image = cv2.imread('image.jpg', 0) # Load as grayscale

if image is None:
    print("Error: Could not load image. Please check the file path.")
else:
    # Get image dimensions
    rows, cols = image.shape

    # Pad the image to optimal size for FFT (power of 2)
    nrows = cv2.getOptimalDFTSize(rows)
    ncols = cv2.getOptimalDFTSize(cols)
    padded_image = np.zeros((nrows, ncols), dtype=np.float32)
    padded_image[:rows, :cols] = image.astype(np.float32)

    # Perform 2D Discrete Fourier Transform (DFT)
    dft = cv2.dft(padded_image, flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    # Create a meshgrid for filter creation
    center_x, center_y = ncols // 2, nrows // 2
    x = np.arange(ncols) - center_x
    y = np.arange(nrows) - center_y
    X, Y = np.meshgrid(x, y)
    D = np.sqrt(X**2 + Y**2) # Euclidean distance from center

    # Butterworth Highpass Filter (BHPF)
    D0 = 50 # Cutoff frequency
    n = 2   # Filter order
    H_bhpf = 1 / (1 + (D0 / D)**(2 * n)) # Note: D0/D for highpass
    # Handle division by zero at the center (D=0) if necessary, typically set to 0
    H_bhpf[D == 0] = 0

    # Apply the filter in the frequency domain
    filtered_dft_shift_bhpf = dft_shift * H_bhpf[:, :, np.newaxis]

    # Compute magnitude spectrum of the filtered image
    magnitude_spectrum_filtered_bhpf = 20 * np.log(cv2.magnitude(filtered_dft_shift_bhpf[:, :, 0], filtered_dft_shift_bhpf[:, :, 1]) + 1)

    # Perform inverse DFT
    f_ishift_bhpf = np.fft.ifftshift(filtered_dft_shift_bhpf)
    img_back_bhpf = cv2.idft(f_ishift_bhpf, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
    img_back_bhpf = img_back_bhpf[:rows, :cols] # Crop back to original size
    img_back_bhpf = np.clip(img_back_bhpf, 0, 255).astype(np.uint8)

    # Display results
    plt.figure(figsize=(15, 5))
    plt.subplot(131), plt.imshow(image, cmap='gray'), plt.title('Original Image')
    plt.subplot(132), plt.imshow(img_back_bhpf, cmap='gray'), plt.title(f'BHPF Sharpened (D0={D0}, n={n})')
    plt.subplot(133), plt.imshow(magnitude_spectrum_filtered_bhpf, cmap='gray'), plt.title('BHPF Magnitude Spectrum')
    plt.tight_layout()
    plt.show()


### Gaussian Highpass Filter (GHPF)

**Description:**
The Gaussian Highpass Filter (GHPF) is derived from the Gaussian Lowpass Filter and offers the smoothest transition in the frequency domain, thus producing no ringing artifacts. It effectively enhances edges and fine details while minimizing artificial oscillations, making it a good choice for natural-looking sharpening.

**Task:**
Implement and apply the Gaussian Highpass Filter in the frequency domain. Show the code, original image, sharpened output, and its magnitude spectrum.

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image
# Replace 'your_image_path.jpg' with the actual path to your image file
image = cv2.imread('image.jpg', 0) # Load as grayscale

if image is None:
    print("Error: Could not load image. Please check the file path.")
else:
    # Get image dimensions
    rows, cols = image.shape

    # Pad the image to optimal size for FFT (power of 2)
    nrows = cv2.getOptimalDFTSize(rows)
    ncols = cv2.getOptimalDFTSize(cols)
    padded_image = np.zeros((nrows, ncols), dtype=np.float32)
    padded_image[:rows, :cols] = image.astype(np.float32)

    # Perform 2D Discrete Fourier Transform (DFT)
    dft = cv2.dft(padded_image, flags=cv2.DFT_COMPLEX_OUTPUT)
    dft_shift = np.fft.fftshift(dft)

    # Create a meshgrid for filter creation
    center_x, center_y = ncols // 2, nrows // 2
    x = np.arange(ncols) - center_x
    y = np.arange(nrows) - center_y
    X, Y = np.meshgrid(x, y)
    D = np.sqrt(X**2 + Y**2) # Euclidean distance from center

    # Gaussian Highpass Filter (GHPF)
    D0 = 50 # Cutoff frequency (standard deviation)
    H_ghpf = 1 - np.exp(-(D**2) / (2 * D0**2)) # 1 - GLPF

    # Apply the filter in the frequency domain
    filtered_dft_shift_ghpf = dft_shift * H_ghpf[:, :, np.newaxis]

    # Compute magnitude spectrum of the filtered image
    magnitude_spectrum_filtered_ghpf = 20 * np.log(cv2.magnitude(filtered_dft_shift_ghpf[:, :, 0], filtered_dft_shift_ghpf[:, :, 1]) + 1)

    # Perform inverse DFT
    f_ishift_ghpf = np.fft.ifftshift(filtered_dft_shift_ghpf)
    img_back_ghpf = cv2.idft(f_ishift_ghpf, flags=cv2.DFT_SCALE | cv2.DFT_REAL_OUTPUT)
    img_back_ghpf = img_back_ghpf[:rows, :cols] # Crop back to original size
    img_back_ghpf = np.clip(img_back_ghpf, 0, 255).astype(np.uint8)

    # Display results
    plt.figure(figsize=(15, 5))
    plt.subplot(131), plt.imshow(image, cmap='gray'), plt.title('Original Image')
    plt.subplot(132), plt.imshow(img_back_ghpf, cmap='gray'), plt.title(f'GHPF Sharpened (D0={D0})')
    plt.subplot(133), plt.imshow(magnitude_spectrum_filtered_ghpf, cmap='gray'), plt.title('GHPF Magnitude Spectrum')
    plt.tight_layout()
    plt.show()
